# Lesson09：Final Project - 我的 AI 夥伴

教學主題：專案整合。

核心目標：自由組合已學技能，完成一個有始有終的任務。

前 20 分鐘 (腦力激盪)：提供幾個專案點子：A. 迎賓機器狗；B. 自動巡邏員；C. 遠端勘探員。

後 100 分鐘 (專案工作坊)：學員自由組隊，選擇一個專案進行開發。分組開發，講師支援，專案
及簡報中必須含一項安全設計及一項倫理考量。

## 9.1 Import and initialize the client for G1

In [ ]:
import time
import sys
from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.go2.sport.sport_client import SportClient
from unitree_sdk2py.g1.loco.g1_loco_client import LocoClient

ChannelFactoryInitialize(0, "ens37")
sport_client = LocoClient()  
sport_client.SetTimeout(10.0)
sport_client.Init()

## 9.2 More actions with arm

In [ ]:
from unitree_sdk2py.g1.arm.g1_arm_action_client import G1ArmActionClient
armAction_client = G1ArmActionClient()  
armAction_client.SetTimeout(10.0)
armAction_client.Init()

In [ ]:
# Define action map
action_map = {
    "two-hand kiss": 11,
    "left kiss": 12,
    "right kiss": 13,
    "hands up": 15,
    "clap": 17,
    "high five": 18,
    "hug": 19,
    "heart": 20,
    "right heart": 21,
    "reject": 22,
    "right hand up": 23,
    "x-ray": 24,
    "face wave": 25,
    "high wave": 26,
    "shake hand": 27,
}

In [ ]:
armAction_client.ExecuteAction(action_map.get("two-hand kiss"))

## 9.3 Design your own robot policy

安全设计方面的提示：

* 参考Lesson07学习的机器狗雷达检测距离。

伦理考量方面提示：

* 参考Lesson04学习的机器狗动作。
* 参考Lesson08-09学习的人形机器人动作。